## Image classification with Convolutional Neural Networks

In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# This file contains all the main external libs we'll use
from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

from pathlib import Path

In [3]:
if os.name == 'nt':
    PATH = str(Path.home()) + "\\data\\dogscats\\"
else:
    PATH = "data/dogscats/"
    
sz=224
arch=vgg16
bs=64

In [4]:
# Uncomment the below if you need to reset your precomputed activations
if os.name == 'nt':
    ## warning: will remove everything under the following path (windows):
    #delPath = f"{PATH}tmp\\"
    #!rmdir /S /Q "{delPath}"
    ;
else:
    #!rm -rf {PATH}tmp
    ;

In [5]:
data = ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch, sz))

In [6]:
learn = ConvLearner.pretrained(arch, data, precompute=True)

In [7]:
%time learn.fit(0.01, 3, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

[ 0.       0.06204  0.03153  0.98892]                                                                                  
[ 1.       0.04266  0.02911  0.9909 ]                                                                                  
[ 2.       0.04116  0.03163  0.98813]                                                                                  

Wall time: 8.63 s


In [8]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)

In [9]:
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs, num_workers=4)
learn = ConvLearner.pretrained(arch, data, precompute=True)

In [10]:
learn.fit(1e-2, 2)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

[ 0.       0.06423  0.03549  0.98695]                                                                                  
[ 1.       0.04845  0.03209  0.98794]                                                                                  



In [11]:
learn.precompute=False

In [12]:
%time learn.fit(1e-2, 1, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[ 0.       0.05348  0.03617  0.98754]                                                                                  

Wall time: 2min 4s


In [13]:
learn.unfreeze()

In [14]:
lr=np.array([1e-4,1e-3,1e-2])

In [15]:
%time learn.fit(lr, 1, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[ 0.       0.05723  0.03262  0.98873]                                                                                  

Wall time: 4min 44s


In [16]:
%time learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=7), HTML(value='')))

[ 0.       0.04096  0.02884  0.99011]                                                                                  
[ 1.       0.03841  0.02523  0.99169]                                                                                  
[ 2.       0.03194  0.02524  0.9915 ]                                                                                  
[ 3.       0.03977  0.0275   0.9915 ]                                                                                  
[ 4.       0.02763  0.02483  0.99169]                                                                                  
[ 5.       0.02321  0.02359  0.99248]                                                                                  
[ 6.       0.02408  0.02327  0.9911 ]                                                                                  

Wall time: 33min 16s


In [17]:
%time learn.fit(lr, 3, cycle_len=3)

HBox(children=(IntProgress(value=0, description='Epoch', max=9), HTML(value='')))

[ 0.       0.0297   0.02501  0.99209]                                                                                  
[ 1.       0.0303   0.02307  0.99229]                                                                                  
[ 2.       0.01943  0.02487  0.9913 ]                                                                                  
[ 3.       0.02883  0.02725  0.9897 ]                                                                                  
[ 4.       0.01703  0.02473  0.99308]                                                                                  
[ 5.       0.0179   0.02254  0.99288]                                                                                  
[ 6.       0.01988  0.02614  0.99089]                                                                                  
[ 7.       0.01763  0.02275  0.99208]                                                                                  
[ 8.       0.01455  0.02147  0.99387]   

In [18]:
%time log_preds_augmented,y = learn.TTA()
log_preds = log_preds_augmented.mean(axis=0)
accuracy(log_preds, y)

Wall time: 1min 28s                                                                                                    


0.993430220983476